In [ ]:
# default_exp linear

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# linear

> Linear functions helpers

In [ ]:
# export 
import numpy as np
from seal import Ciphertext, Plaintext, CKKSEncoder, DoubleVector, Evaluator, GaloisKeys
from typing import List

In [ ]:
# export
def pad_along_axis(array: np.ndarray, target_length, axis=0):
    """Pads an array to a given target length, on a given axis."""
    pad_size = target_length - array.shape[axis]
    axis_nb = len(array.shape)

    if pad_size <= 0:
        return array

    npad = [(0, 0) for x in range(axis_nb)]
    npad[axis] = (0, pad_size)

    b = np.pad(array, pad_width=npad, mode='constant', constant_values=0)

    return b

In [ ]:
# export
def arrays_to_ptx(arrays: List[np.ndarray], encoder: CKKSEncoder, scale: float) -> List[Plaintext]:
    """Converts an array of arrays to an array of plaintexts."""
    outputs = []
    for array in arrays:
        ptx = Plaintext()
        encoder.encode(DoubleVector(list(array)), scale, ptx)
        outputs.append(ptx)
    return outputs

In [ ]:
# export
def extract_diagonals(matrix: np.ndarray) -> List[np.ndarray]:
    """Extracts the diagonals of the matrix"""
    assert matrix.shape[0] == matrix.shape[1], "Non square matrix"
    dim = matrix.shape[0]
    
    diagonals = []
    
    for i in range(dim):
        diagonal = []
        for j in range(dim):
            diagonal.append(matrix[j][(j+i) % dim])
        diagonal = np.array(diagonal)
        diagonals.append(diagonal)
    return diagonals

In [ ]:
# export
def matrix_multiply_diagonals(diagonals: List[Plaintext], ctx: Ciphertext, 
                              evaluator: Evaluator, galois_keys: GaloisKeys):
    output = Ciphertext()
    
    for i in range(len(diagonals)):
        
        temp = Ciphertext()
        diagonal = diagonals[i]
        
        evaluator.rotate_vector(ctx, i, galois_keys, temp)
            
        evaluator.mod_switch_to_inplace(diagonal, temp.parms_id())
        evaluator.multiply_plain_inplace(temp, diagonal)
        evaluator.rescale_to_next_inplace(temp)
        
        if i == 0:
            output = temp
        else:
            evaluator.add_inplace(output, temp)

    return output